In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [6]:
era5_uu = xr.open_dataset(r'./merge_UU_1990-2020.nc')
era5_vv = xr.open_dataset(r'./merge_VV_1990-2020.nc')
era5_p = xr.open_dataset(r'./merge_p_1990-2020.nc')
# wind_x

In [14]:
era5_p

<xarray.Dataset> Size: 2GB
Dimensions:       (time: 271752, rlat: 27, rlon: 39)
Coordinates:
  * time          (time) datetime64[ns] 2MB 1990-01-01T01:00:00 ... 2021-01-01
    lon           (rlat, rlon) float64 8kB ...
    lat           (rlat, rlon) float64 8kB ...
  * rlon          (rlon) float64 312B -5.5 -5.39 -5.28 ... -1.54 -1.43 -1.32
  * rlat          (rlat) float64 216B -5.36 -5.25 -5.14 ... -2.72 -2.61 -2.5
Data variables:
    rotated_pole  int32 4B ...
    slp           (time, rlat, rlon) float64 2GB ...
Attributes:
    CDI:          Climate Data Interface version 2.4.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.0
    institution:  Le nom de votre institution
    title:        PC_011_E5_ND /lustre06/project/6055357/yboyuan/gem_to_netcd...
    history:      Wed Apr 30 11:30:29 2025: cdo sellonlatbox,-117.3,-108.9,60...
    comments:     Bla bla bla cdf2ccc 1.5
    reference:    http://www.votre.site.web
    CDO:          Climate Data Operators version 2.4.0 (https://mpimet.mpg.de...

In [7]:
# time_count = len(wind_x.time)
# time_count
len(era5_uu.time)

271752

In [4]:
time_range = pd.date_range("2000-09-18 00", "2000-09-23 00", freq="1h")
print(len(time_range))
time_count = len(time_range)

121


In [8]:
p_data =  era5_p.slp.sel(time=time_range[0], method='nearest')
np.shape(p_data)

(27, 39)

In [15]:
lat_era5 = era5_p["lat"].values
lon_era5 = era5_p["lon"].values

In [16]:
lat_era5

array([[60.15431595, 60.17339706, 60.19210815, ..., 60.60163116,
        60.60691452, 60.61180496],
       [60.26259613, 60.2817421 , 60.30051804, ..., 60.71149445,
        60.71679688, 60.72170639],
       [60.37086105, 60.39007568, 60.40891647, ..., 60.82135391,
        60.82667542, 60.831604  ],
       ...,
       [62.74953842, 62.7703476 , 62.79075241, ..., 63.23802948,
        63.24380875, 63.24916077],
       [62.85750198, 62.87838745, 62.89886856, ..., 63.34786606,
        63.35366821, 63.35903931],
       [62.96544647, 62.98641586, 63.00697327, ..., 63.45770264,
        63.46352386, 63.46891785]])

In [22]:
import os
from scipy.interpolate import griddata
save_file = './wind_field_2000-09-18_09-23.txt'
# os.system("rm -rf ./wind_field.txt") # in Linux
os. remove(save_file) # in Windows

lon_llc = -117.3
lat_llc = 60.6
lon_urc = -108.9
lat_urc = 63.1
delta_lon = 0.1
delta_lat = 0.1
lon_count = np.int32(np.abs(lon_llc-lon_urc)/delta_lon+1)
lat_count = np.int32(np.abs(lat_llc-lat_urc)/delta_lat+1)

# write the header first
with open(save_file, "a") as f:
    f.write('"Title" "File Title"')
    f.write('\n')
    f.write('"Dim" 2')
    f.write('\n')
    f.write('"Geo" "LONG/LAT" %g %g 0'% ((lon_llc), (lat_llc)))
    f.write('\n')
    f.write('"Time" "EqudistantTimeAxis" "2000-09-18" "00:00:00" 121 3600') # need to modify here for time
    f.write('\n')
    f.write('"NoGridPoints" %s %s'% (str(lon_count), str(lat_count)))
    f.write('\n')
    f.write('"Spacing" %g %g'%(delta_lon, delta_lat))
    f.write('\n')
    f.write('"NoStaticItems" 0')
    f.write('\n')
    f.write('"NoDynamicItems" 3') # need to modify here for items
    f.write('\n')
    f.write('"Item" "MeanSeaLevelPressure" "Pressure" "Pascal"')
    f.write('\n')
    f.write('"Item" "U-wind" "Wind Velocity" "m/s"')
    f.write('\n')
    f.write('"Item" "V-wind" "Wind Velocity" "m/s"')
    f.write('\n')
    f.write('NoCustomBlocks 1')
    f.write('\n')
    f.write('"M21_Misc" 1 7 0 -1E-030 -900 -999 -1E-030 -1E-030 -1E-030')
    f.write('\n')
    f.write('"Delete" -1E-030')
    f.write('\n')
    f.write('"DataType" 0')
    f.write('\n')
    f.write('\n')

with open(save_file, "a") as f:
    # no "preamber" for now
    # loop over the layers
    for itime in range(time_count):
    # for itime in range(2):
        output_tot = np.zeros((1, lon_count))
        uu_data =  era5_uu.UU.sel(time=time_range[itime], method='nearest')
        vv_data =  era5_vv.VV.sel(time=time_range[itime], method='nearest')
        p_data =  era5_p.slp.sel(time=time_range[itime], method='nearest')
        lat_interp = np.zeros([lon_count,lat_count])
        lon_interp = np.zeros([lon_count,lat_count])
        # construction of the wind matrix
        for ilon in range(lon_count):
            for ilat in range(lat_count):
                lat_interp[ilon,ilat] = lat_llc+delta_lat*(lat_count-1-ilat)
                lon_interp[ilon,ilat] = lon_llc+delta_lon*(ilon)
        grid_out = np.concatenate([lon_interp.reshape(-1,1), lat_interp.reshape(-1,1)], axis=1)
        grid_in = np.concatenate([lon_era5.reshape(-1,1), lat_era5.reshape(-1,1)], axis=1)

        u10_out = griddata(grid_in, np.array(uu_data).flatten(), grid_out, method='linear')
        v10_out = griddata(grid_in, np.array(vv_data).flatten(), grid_out, method='linear')
        sp_out = griddata(grid_in, np.array(p_data).flatten(), grid_out, method='linear')/100.0

        u10_out = u10_out.reshape(lon_count,lat_count)
        v10_out = v10_out.reshape(lon_count,lat_count)
        sp_out = sp_out.reshape(lon_count,lat_count)

        output_tot = np.zeros((1, lon_count))
        f.write('"tstep" %s "item" 1 "layer" 0 \n'%str(np.int64(itime)))
        # item 1
        for j in range(lat_count):
            for k in range(lon_count):
                # output_tot[:, k] = sp_out[(lat_count-j-1), k]*100.0
                output_tot[:, k] = sp_out[k,j]*100.0
            # output_tot = np.transpose(output_tot)
            np.savetxt(f, output_tot, fmt='%g')
        f.write('\n')
        f.write('"tstep" %s "item" 2 "layer" 0 \n'%str(np.int64(itime)))
        # item 2
        for j in range(lat_count):
            for k in range(lon_count):
                output_tot[:, k] = u10_out[k,j]
            # output_tot = np.transpose(output_tot)
            np.savetxt(f, output_tot, fmt='%g')
        f.write('\n')
        # item 3
        f.write('"tstep" %s "item" 3 "layer" 0 \n'%str(np.int64(itime)))
        for j in range(lat_count):
            for k in range(lon_count):
                output_tot[:, k] = v10_out[k,j]
            np.savetxt(f, output_tot, fmt='%g')
        f.write('\n')
        print('Time'+': '+str(time_range[itime]))

f.close()

Time: 2000-09-18 00:00:00
Time: 2000-09-18 01:00:00
Time: 2000-09-18 02:00:00
Time: 2000-09-18 03:00:00
Time: 2000-09-18 04:00:00
Time: 2000-09-18 05:00:00
Time: 2000-09-18 06:00:00
Time: 2000-09-18 07:00:00
Time: 2000-09-18 08:00:00
Time: 2000-09-18 09:00:00
Time: 2000-09-18 10:00:00
Time: 2000-09-18 11:00:00
Time: 2000-09-18 12:00:00
Time: 2000-09-18 13:00:00
Time: 2000-09-18 14:00:00
Time: 2000-09-18 15:00:00
Time: 2000-09-18 16:00:00
Time: 2000-09-18 17:00:00
Time: 2000-09-18 18:00:00
Time: 2000-09-18 19:00:00
Time: 2000-09-18 20:00:00
Time: 2000-09-18 21:00:00
Time: 2000-09-18 22:00:00
Time: 2000-09-18 23:00:00
Time: 2000-09-19 00:00:00
Time: 2000-09-19 01:00:00
Time: 2000-09-19 02:00:00
Time: 2000-09-19 03:00:00
Time: 2000-09-19 04:00:00
Time: 2000-09-19 05:00:00
Time: 2000-09-19 06:00:00
Time: 2000-09-19 07:00:00
Time: 2000-09-19 08:00:00
Time: 2000-09-19 09:00:00
Time: 2000-09-19 10:00:00
Time: 2000-09-19 11:00:00
Time: 2000-09-19 12:00:00
Time: 2000-09-19 13:00:00
Time: 2000-0